<a href="https://colab.research.google.com/github/eentartetekunst/NGS_bioinformatics/blob/main/HW1_Hisat2_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
! pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install -y -c bioconda samtools

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.

Retrieving notices: ...working... done


In [ ]:
! sudo apt install bedtools 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bedtools is already the newest version (2.26.0+dfsg-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
! conda install -c bioconda regtools 

Solving environment: \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.

Retrieving notices: ...working... done


## import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir reads

In [ ]:
! mv **/*.fq.gz /content/drive/MyDrive/rna.seq.data reads

mv: cannot stat '**/*.fq.gz': No such file or directory


In [ ]:
! ls reads/rna.seq.data

B14.5.fq.gz  B17.5.fq.gz  B34.fq.gz    C15.5.fq.gz  C20.fq.gz
B15.5.fq.gz  B20.fq.gz	  C14.5.fq.gz  C17.5.fq.gz  C34.fq.gz


## 2) install hisat2

In [ ]:
! sudo apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,164 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://sec

In [ ]:
! sudo apt-get -y install hisat2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  hisat2
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 1,278 kB of archives.
After this operation, 5,144 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 hisat2 amd64 2.1.0-1 [1,278 kB]
Fetched 1,278 kB in 1s (1,716 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-pre

## 3) download chr19 sequence and its annotation

In [ ]:
! wget http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.chromosome.19.fa.gz #chr19 sequence
! wget http://ftp.ensembl.org/pub/release-107/gtf/mus_musculus/Mus_musculus.GRCm39.107.gtf.gz # annotation
! gzip -d Mus_musculus.GRCm39.dna.chromosome.19.fa.gz
! gzip -d Mus_musculus.GRCm39.107.gtf.gz

--2022-10-11 13:26:45--  http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.chromosome.19.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17732438 (17M) [application/x-gzip]
Saving to: ‘Mus_musculus.GRCm39.dna.chromosome.19.fa.gz’

Mus_musculus.GRCm39 100%[===================>]  16.91M  4.68MB/s    in 3.6s    

2022-10-11 13:26:49 (4.66 MB/s) - ‘Mus_musculus.GRCm39.dna.chromosome.19.fa.gz’ saved [17732438/17732438]

--2022-10-11 13:26:49--  http://ftp.ensembl.org/pub/release-107/gtf/mus_musculus/Mus_musculus.GRCm39.107.gtf.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31861821 (30M) [application/x-gzip]
Saving to: ‘Mus_musculus.GRCm39.107.gtf.gz’

## 4) select chr 19 from the annotation

In [ ]:
! touch chr19-annotation 
! grep -P '^19\t' /content/Mus_musculus.GRCm39.107.gtf > chr19-annotation 

In [ ]:
!ls -lh

total 918M
-rw-r--r-- 1 root root  27M Oct 11 13:27 chr19-annotation
-rw-r--r-- 1 root root  16K Oct 11 13:09 condacolab_install.log
drwx------ 5 root root 4.0K Oct 11 13:26 drive
-rw-r--r-- 1 root root 831M Jun  4 09:40 Mus_musculus.GRCm39.107.gtf
-rw-r--r-- 1 root root  60M Jun  4 08:49 Mus_musculus.GRCm39.dna.chromosome.19.fa
drwxr-xr-x 3 root root 4.0K Oct 11 13:26 reads
drwxr-xr-x 1 root root 4.0K Oct  7 13:35 sample_data


In [ ]:
! head -n2 chr19-annotation 

19	ensembl	gene	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA";
19	ensembl	transcript	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; transcript_id "ENSMUST00000104035"; transcript_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA"; transcript_name "Gm22271-201"; transcript_source "ensembl"; transcript_biotype "snoRNA"; tag "basic"; transcript_support_level "NA (assigned to previous version 1)";


## 5) build index for chr 19

## map all reads on chr19 sequence (no trimming, known splice sites)
use --known-splicesite-infile:
With this mode, you can provide a list of known splice sites, which HISAT makes use of to align reads with small anchors.
You can create such a list using python extract_splice_sites.py genes.gtf > splicesites.txt, where extract_splice_sites.py is included in the HISAT package, genes.gtf is a gene annotation file, and splicesites.txt is a list of splice sites with which you provide HISAT in this mode

## install extract_splice_sites.py and extract_exons.py

In [ ]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_splice_sites.py
! chmod a+x extract_splice_sites.py
! ./extract_splice_sites.py

--2022-10-11 13:27:04--  https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_splice_sites.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5021 (4.9K) [text/plain]
Saving to: ‘extract_splice_sites.py’

extract_splice_site 100%[===================>]   4.90K  --.-KB/s    in 0s      

2022-10-11 13:27:04 (35.9 MB/s) - ‘extract_splice_sites.py’ saved [5021/5021]

usage: extract_splice_sites.py [-h] [-v] [gtf_file]

Extract splice junctions from a GTF file

positional arguments:
  gtf_file       input GTF file (use "-" for stdin)

optional arguments:
  -h, --help     show this help message and exit
  -v, --verbose  also print some statistics to stderr


In [ ]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_exons.py
! chmod a+x extract_exons.py
! ./extract_exons.py

--2022-10-11 13:27:05--  https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_exons.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5597 (5.5K) [text/plain]
Saving to: ‘extract_exons.py’

extract_exons.py    100%[===================>]   5.47K  --.-KB/s    in 0s      

2022-10-11 13:27:05 (58.3 MB/s) - ‘extract_exons.py’ saved [5597/5597]

usage: extract_exons.py [-h] [-v] [gtf_file]

Extract exons from a GTF file

positional arguments:
  gtf_file       input GTF file (use "-" for stdin)

optional arguments:
  -h, --help     show this help message and exit
  -v, --verbose  also print some statistics to stderr


## extract splice sites and exon's locations  from the annotation

In [ ]:
! ./extract_splice_sites.py chr19-annotation > chr19.ss # extract sites from the annotation
! ! ./extract_exons.py chr19-annotation > chr19.exons 

In [ ]:
! head -n2 chr19.ss 

19	3103277	3109879	-
19	3109990	3119366	-


In [ ]:
! head -n2 chr19.exons

19	3103070	3103277	-
19	3109879	3109990	-


## map reads on chr19 with no trimming, with known splice sites

---



## map b14.5 on chr19

In [ ]:
! hisat2-build --ss chr19.ss --exon chr19.exons Mus_musculus.GRCm39.dna.chromosome.19.fa base-index

Settings:
  Output files: "base-index.*.ht2"
  Line rate: 7 (line is 128 bytes)
  Lines per side: 1 (side is 128 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  Mus_musculus.GRCm39.dna.chromosome.19.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
  Time to read SNPs and splice sites: 00:00:00
Generation 0 (58229592 -> 58229592 nodes, 0 ranks)
COUNTED NEW NODES: 0
COUNTED TEMP NODES: 0
RESIZED NODES: 0
RESIZED NODES: 0
MADE NEW NODES: 0
Generation 1 (582388

In [ ]:
i='B20'
index='base-index'
fq='/content/reads/rna.seq.data'

In [ ]:
# redirect stdout of hisat2 (sam alignment file) to samtools view so that we only save compressed file 
! hisat2 --summary-file $i\.log --no-softclip --no-unal -x $index -U $fq/$i\.fq.gz | samtools view -Sb - > $i\.bam

599777 reads; of these:
  599777 (100.00%) were unpaired; of these:
    51160 (8.53%) aligned 0 times
    538522 (89.79%) aligned exactly 1 time
    10095 (1.68%) aligned >1 times
91.47% overall alignment rate


## 6) Прокартируйте все fq файлы (начните с одного) на 19ую хромосому при помощи hisat2 не допуская обрезания ридов и сообщив hisat2 координаты сайтов сплайсинга 

In [ ]:
%%shell 
reads_list='C14.5 B34 B14.5 C15.5 C17.5 B17.5 C34 B20 B15.5 C20' 
index='base-index'
fq='/content/reads/rna.seq.data'

for i in ${reads_list}
do hisat2 --summary-file ${i}.log --no-softclip --no-unal -x ${index} -U ${fq}/${i}.fq.gz | samtools view -Sb - > ${i}.bam 
done

599461 reads; of these:
  599461 (100.00%) were unpaired; of these:
    49649 (8.28%) aligned 0 times
    538759 (89.87%) aligned exactly 1 time
    11053 (1.84%) aligned >1 times
91.72% overall alignment rate
599683 reads; of these:
  599683 (100.00%) were unpaired; of these:
    51939 (8.66%) aligned 0 times
    539715 (90.00%) aligned exactly 1 time
    8029 (1.34%) aligned >1 times
91.34% overall alignment rate
599672 reads; of these:
  599672 (100.00%) were unpaired; of these:
    47202 (7.87%) aligned 0 times
    540491 (90.13%) aligned exactly 1 time
    11979 (2.00%) aligned >1 times
92.13% overall alignment rate
600324 reads; of these:
  600324 (100.00%) were unpaired; of these:
    75959 (12.65%) aligned 0 times
    514263 (85.66%) aligned exactly 1 time
    10102 (1.68%) aligned >1 times
87.35% overall alignment rate
600268 reads; of these:
  600268 (100.00%) were unpaired; of these:
    59115 (9.85%) aligned 0 times
    531032 (88.47%) aligned exactly 1 time
    10121 (1.69

## 7) Выберите случайно один образец

## 8)Сколько ридов картируется в регион 19:12485000-12490000 в этом образце? 

In [ ]:
! samtools sort -o B20.s.bam B20.bam #coordinate sorting

In [ ]:
! samtools index B20.s.bam # Generate BAI-format index for BAM files

In [ ]:
! samtools view B20.s.bam 19:12485000-12490000 

HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479312	0	42M56847N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479314	0	42M56845N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479316	0	42M56843N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHD

In [168]:
! samtools view B20.s.bam 19:12485000-12490000 
# эти 7  ридов картируются на данный участок 

HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479312	0	42M56847N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479314	0	42M56845N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479316	0	42M56843N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHD

## 9) Сколько из них картируются только в одно место генома?


In [ ]:
! samtools view B20.s.bam 19:12485000-12490000 -d NH:1 -c  
#  риды, которые картировлаись на одно место в данном участке 
# -c Instead of printing the alignments, only print the total number. 

3


In [171]:
! samtools view B20.s.bam 19:12485000-12490000  -d NH:1 

HISEQ:129:C2LRCACXX:1:2108:20530:65021	16	19	12486735	60	101M	*	0	0	TTTAAAATCACCCAATAGACTGACTTGCCAGCGTGCCTGTAAGGAGTTATCTAGAATCAGTTAACTAAGGTAGGAATACCCACCCTAACCTTGGGCAGCAC	:CCDEEDB@@DCDEECDCCDEDCBC@DDFFHHHIIIIIIIIIIIIIIIIIFIIIIIIIIIIIIIIIIIIIIIIIHE<?IGF?IIIIGGHHHHHFFFFFCCC	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	NH:i:1
HISEQ:129:C2LRCACXX:1:1204:10203:87648	16	19	12487196	60	101M	*	0	0	TGTATTGACATCTTAAACTATTTCAAATATACCCCAACCTCAGCTAATCTGGGTTACAAACCCCCTAAAATTCAAATAACATCTCATTTGAATGTCCAGAC	@C:EEEDEDDDCDEEFCDDDDDDEFFFDDBBDDFFECHHHIJJJJJJJJJJJJIIJJIHFIJJJJIJJJJIIJJJJIIJJIHHIJJJJHHHHHFFFFFCCC	AS:i:-5	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:5C95	YT:Z:UU	NH:i:1
HISEQ:129:C2LRCACXX:1:2104:14666:2583	16	19	12487196	60	101M	*	0	0	TGTATTGACATCTTAAACTATTTCAAATATACCCCAACCTCAGCTAATCTGGGTTACAAACCCCCTAAAATTCAAATAACATCTCATTTGAATGTCCAGAC	CDEEEEDEDDDDDEEDDDDDDDEEFFFEDBDDDFFEHHHHJJJJJJJIJJJJJJJJIHHFJJJJIJJJJIIHIJJJJJHIIHJIJJJJHHHHHFFFFFCCC	AS:i:-5	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:5C9

## 10) Сколько ридов картровалось без замен? Сколько с 1, 2 и т. д. заменами?

In [174]:
! samtools view B20.s.bam 19:12485000-12490000 -d NM:1 -c # с 1 заменой 

2


In [ ]:
! samtools view B20.s.bam 19:12485000-12490000  -d NM:2 -c # с 2 заменами

0


Риды, картировавшиеся на всю хромосому с i заменами:

In [ ]:
%%shell
list_my_i='1 2 3 4 5 6 7 8 9 10 11 12'
for i in ${list_my_i}
do samtools view B14.5.s.bam  -d NM:${i} -c # с i заменами -- картирование на всю хромасому
done

84502
20990
9620
2759
1451
771
344
125
52
16
0
0


## 11) Сколько ридов картировалось на экзон-экзонные границы? 

https://regtools.readthedocs.io/en/latest/commands/junctions-extract/ 

The junctions extract command can be used to extract exon-exon junctions from an RNAseq BAM file. The output is a BED file in the BED12 format

In [ ]:
! touch B20.s.cut.bam

In [ ]:
! cat B20.s.bam 19:12485000-12490000  > B20.s.cut.bam 

In [ ]:
! samtools index B20.s.cut.bam 

[W::bam_hdr_read] EOF marker is absent. The input is probably truncated
[E::bgzf_read_block] Invalid BGZF header at offset 35331189
[E::bgzf_read] Read block operation failed with error 2 after 0 of 4 bytes
samtools index: failed to create index for "B20.s.cut.bam": Invalid argument


In [ ]:
# /content/B14.5.bam.bai path to coordinate-sorted file
! regtools junctions extract -o B20.bed -s XS B20.s.bam 


Program:	regtools
Version:	0.5.2
Minimum junction anchor length: 8
Minimum intron length: 70
Maximum intron length: 500000
Alignment: B20.s.bam
Output file: B20.bed



In [ ]:
! head B20.bed

In [ ]:
! regtools junctions annotate -o B20.exonexon.reads B20.bed Mus_musculus.GRCm39.dna.chromosome.19.fa chr19-annotation 

In [ ]:
! head B20.exonexon.reads 
# score - The number of reads supporting the junction. [integer]

chrom	start	end	name	score	strand	splice_site	acceptors_skipped	exons_skipped	donors_skipped	anchor	known_donor	known_acceptor	known_junction	gene_names	gene_ids	transcripts
19	3133341	3137331	JUNC00000001	1	+	GT-AG	0	0	0	N	0	0	0	NA	NA	NA
19	3277818	3277895	JUNC00000002	1	+	GT-AG	0	0	0	N	0	0	0	NA	NA	NA
19	3311635	3311975	JUNC00000003	2	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292
19	3312144	3314811	JUNC00000004	2	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292
19	3315665	3316336	JUNC00000005	4	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000143079,ENSMUST00000154537
19	3316459	3317242	JUNC00000006	3	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000143079,ENSMUST00000154537
19	3317336	3318298	JUNC00000007	1	-	GT-AG	1	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST000

In [ ]:
! cat B20.exonexon.reads | awk '($2 <= 12490000 && $3 >= 12485000)' 
# риды из данного интервала, в которых есть экзон-экзонная граница

19	12479353	12536201	JUNC00002608	1	+	TC-GT	1	0	2	N	0	0	0	NA	NA	NA
19	12479355	12536201	JUNC00002610	1	+	TC-GT	1	0	2	N	0	0	0	NA	NA	NA
19	12479357	12536201	JUNC00002611	1	+	TC-GT	1	0	2	N	0	0	0	NA	NA	NA
19	12479359	12536201	JUNC00002612	1	+	TG-GT	1	0	2	N	0	0	0	NA	NA	NA


109117 reads mapped over exon exon junctions

## 12) Перечислите координаты всех интронов в данном интервале подтверждённых хотя бы одним ридом в формате: chr:from-to coverage

In [ ]:
! samtools view B20.s.bam -b 19:12485000-12490000 > targetB20.s.bam # choose target region

In [ ]:
! bedtools bamtobed -i targetB20.s.bam > targetB20.s.bed 

In [ ]:
! head targetB20.s.bed # reads in target region 

19	12479311	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479313	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479315	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479317	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12486734	12486835	HISEQ:129:C2LRCACXX:1:2108:20530:65021	60	-
19	12487195	12487296	HISEQ:129:C2LRCACXX:1:1204:10203:87648	60	-
19	12487195	12487296	HISEQ:129:C2LRCACXX:1:2104:14666:2583	60	-


### bedtools complement -i exons.bed -g genome.txt > non-exonic.bed -- to get intron coordinates 

In [ ]:
! head /content/chr19.exons

19	3103070	3103277	-
19	3109879	3109990	-
19	3115710	3118148	-
19	3119366	3119439	-
19	3122687	3122952	-
19	3125713	3125884	-
19	3184644	3184958	+
19	3186983	3187089	-
19	3192759	3193064	+
19	3195867	3196264	-


Если вычести из всех ридов те, что попадают на экзоны, мы получим покрытие интронов:

In [ ]:
! bedtools subtract -a targetB20.s.bed -b chr19.exons 

Координаты интронных ридов:

In [ ]:
! bedtools subtract -a targetB20.s.bed -b chr19.exons  > introns.bed 
! awk -F"\t" '$5 > 0 { print $2"\t"$3 }' introns.bed 
# retrieve introns in the region, covered by at least one read 
# $5 > 0 stands for coverage score

12486734	12486835
12487195	12487296
12487195	12487296
